# `Jai Chaudhry 2K18-SE-069`

## Experiment - 9

- Build an Artificial Neural Network by implementing the Backpropagation algorithm and test the
  same using appropriate data sets


In [2]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
import copy 


In [106]:
def tanh(x, derivative=False):
    if derivative:
        1-x**2
    return np.tanh(x)


def sigmoid(x, derivative=False):
    if derivative:
        return x*(1-x)
    return 1/(1+np.exp(-x + eps))


def mse(y_pred, y, derivative=False):
    if derivative:
        return y_pred*(y_pred-y)
    return 0.5*(y_pred-y)**2



In [3]:
# Get Numbers in a range here (-1,1) , then specify shape
a = np.random.uniform(-1,1, (2,2))
print(a)


[[ 0.05172371 -0.03248593]
 [ 0.75537272 -0.23551374]]


In [108]:
class Layer:
    def __init__(self, weights_shape, previous_result=None):
        self.previous_result = previous_result
        self.weights = self.init_weights(weights_shape)
        self.out = None
        self.bias = np.random.uniform(-1,1)

    def init_weights(self, weights_shape):
        return np.random.uniform(-1,1, weights_shape)


    def compute_output(self, previous_result, activation_function):
        self.previous_result = previous_result
        self.out = activation_function(self.previous_result.dot(self.weights) + self.bias)


In [152]:
class ANN:
    def __init__(self, input_values, output_values, learning_rate=1e-1, activation_function = sigmoid):
        self.input_values = input_values
        self.layers = []
        self.output_values = output_values
        self.lr = learning_rate
        self.activation_function = activation_function


    def add_layer(self, nb_nodes):
        if len(self.layers) == 0:
            weights_shape = (self.input_values.shape[1], nb_nodes)
            previous_result = self.input_values
            
            layer = Layer(weights_shape, previous_result)
            
        else:
            weights_shape = (self.layers[-1].weights.shape[1], nb_nodes)
            layer = Layer(weights_shape, self.layers[-1].out)
            
        self.layers.append(layer)


    def feed_forward(self):
        previous_result = self.layers[0].previous_result
        
        for layer in self.layers:
            layer.compute_output(previous_result, self.activation_function)
            previous_result = layer.out


    def back_prop(self):
        previous_layer = last_layer = self.layers[-1]
        
        d = mse(last_layer.out, self.output_values, True) * self.activation_function(last_layer.out, True)
        
        last_layer.weights -= self.lr * last_layer.previous_result.T.dot(d)
        last_layer.bias -= self.lr * np.mean(d)
        
        for layer in np.flip(self.layers, axis=0)[1:]:
            d = d.dot(previous_layer.weights.T) * self.activation_function(layer.out, True)
            layer.weights -= self.lr * layer.previous_result.T.dot(d)
            layer.bias -= self.lr * np.mean(d)
            previous_layer = layer


    def train(self, nb_epochs = 65000):
        self.layers = np.array(self.layers)
        error_list = []
        
        for i in range(nb_epochs):
            self.feed_forward()
            self.back_prop()
            error_list.append(np.mean(mse(self.layers[-1].out, self.output_values)))
            if i%5000==0:
                print(i)
        return error_list
    
    
    def predict(self, input_values):
        previous_result = np.array(input_values)
        
        for layer in self.layers:
            layer.compute_output(previous_result, self.activation_function)
            previous_result = layer.out
        return previous_result



In [99]:
train = np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
])


train_result = np.array([0,1,1,0]).reshape((-1,1))
ann = ANN(train, train_result, learning_rate=1e-1, activation_function = sigmoid)
ann.add_layer(10)
ann.add_layer(5)
ann.add_layer(3)

ann.add_layer(1)
error = ann.train(nb_epochs=65000)


In [100]:
print(np.around(ann.predict(train)),"\n")

print(ann.predict(train))


[[0.]
 [1.]
 [1.]
 [0.]] 

[[0.04425457]
 [0.99049105]
 [0.99303464]
 [0.03966538]]


# `Jai Chaudhry 2K18-SE-069`